# Azure AI Agentleri Model Context Protocol (MCP) Desteği ile - Python

Bu not defteri, Python'da Model Context Protocol (MCP) araçlarıyla Azure AI Agentlerini nasıl kullanacağınızı gösterir. Anahtarsız kimlik doğrulama kullanarak gelişmiş yetenekler için harici MCP sunucularını (örneğin, Microsoft Learn) kullanabilen akıllı bir ajan oluşturmayı açıklamaktadır.


## Gerekli Python Paketlerini Yükleyin

Öncelikle, gerekli Python paketlerini yüklememiz gerekiyor:
- **azure-ai-projects**: Temel Azure AI Projects SDK'sı
- **azure-ai-agents**: Ajanlar oluşturmak ve yönetmek için Azure AI Agents SDK'sı
- **azure-identity**: DefaultAzureCredential kullanarak anahtarsız kimlik doğrulama sağlar
- **mcp**: Python için Model Context Protocol uygulaması


## Anahtarsız Kimlik Doğrulama Faydaları

Bu not defteri, **anahtarsız kimlik doğrulama**yı gösterir ve aşağıdaki avantajları sağlar:
- ✅ **Yönetilecek API anahtarı yok** - Azure kimlik tabanlı kimlik doğrulama kullanır
- ✅ **Gelişmiş güvenlik** - Kodda veya yapılandırma dosyalarında gizli bilgiler saklanmaz
- ✅ **Otomatik kimlik bilgisi yenileme** - Azure kimlik bilgisi yaşam döngüsü yönetimini üstlenir
- ✅ **Rol tabanlı erişim kontrolü** - Hassas izinler için Azure RBAC kullanır
- ✅ **Çoklu ortam desteği** - Geliştirme ve üretim ortamlarında sorunsuz çalışır

`DefaultAzureCredential` otomatik olarak en uygun kimlik bilgisi kaynağını seçer:
1. **Yönetilen Kimlik** (Azure'da çalışırken)
2. **Azure CLI** kimlik bilgileri (yerel geliştirme sırasında)
3. **Visual Studio** kimlik bilgileri
4. **Ortam değişkenleri** (eğer yapılandırılmışsa)
5. **Etkileşimli tarayıcı** kimlik doğrulama (son çare olarak)


## Anahtarsız Kimlik Doğrulama Kurulumu

**Anahtarsız kimlik doğrulama için ön koşullar:**

### Yerel Geliştirme İçin:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Azure Ortamları İçin:
- Azure kaynağınızda **Sistem Atamalı Yönetilen Kimlik** özelliğini etkinleştirin
- Yönetilen kimliğe uygun **RBAC rolleri** atayın:
  - Azure OpenAI erişimi için `Cognitive Services OpenAI User`
  - Azure AI Projeleri erişimi için `AI Developer`

### Ortam Değişkenleri (İsteğe Bağlı):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**API anahtarlarına veya bağlantı dizelerine gerek yok!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Gerekli Kütüphaneleri İçe Aktar

Gerekli Python modüllerini içe aktarın:
- **os, time**: Ortam değişkenleri ve gecikmeler için standart Python kütüphaneleri
- **AIProjectClient**: Azure AI Projeleri için ana istemci
- **DefaultAzureCredential**: Azure hizmetleri için anahtarsız kimlik doğrulama
- **MCP ile ilgili sınıflar**: MCP araçları oluşturmak, yönetmek ve onayları işlemek için


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## MCP Sunucu Ayarlarını Yapılandırma

MCP sunucu yapılandırmasını, varsayılan değerlerle geri dönüş sağlayan ortam değişkenlerini kullanarak ayarlayın:
- **MCP_SERVER_URL**: MCP sunucusunun URL'si (varsayılan olarak Microsoft Learn API)
- **MCP_SERVER_LABEL**: MCP sunucusunu tanımlamak için bir etiket (varsayılan olarak "mslearn")

Bu yaklaşım, farklı ortamlar arasında esnek bir yapılandırma sağlar.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Azure AI Proje İstemcisi Oluşturma (Anahtarsız Kimlik Doğrulama)

Azure AI Proje istemcisini **anahtarsız kimlik doğrulama** kullanarak başlatın:
- **endpoint**: Azure AI Foundry proje uç nokta URL'si
- **credential**: Güvenli, anahtarsız kimlik doğrulama için `DefaultAzureCredential()` kullanır
- **API anahtarlarına gerek yok**: En uygun kimlik bilgilerini otomatik olarak bulur ve kullanır

**Kimlik Doğrulama Akışı:**
1. Yönetilen Kimlik (Azure ortamlarında) kontrol edilir
2. Yerel geliştirme için Azure CLI kimlik bilgilerine geçiş yapar
3. Gerekirse diğer mevcut kimlik bilgisi kaynaklarını kullanır

Bu yaklaşım, kodunuzda API anahtarlarını veya bağlantı dizelerini yönetme ihtiyacını ortadan kaldırır.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## MCP Araç Tanımı Oluşturma

Microsoft Learn MCP sunucusuna bağlanan bir MCP aracı oluşturun:
- **server_label**: MCP sunucusu için tanımlayıcı
- **server_url**: MCP sunucusunun URL uç noktası
- **allowed_tools**: Hangi araçların kullanılabileceğini sınırlamak için isteğe bağlı bir liste (boş liste tüm araçlara izin verir)

Bu araç, ajanın Microsoft Learn dokümantasyonuna ve kaynaklarına erişmesini sağlayacaktır.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Temsilci Oluştur ve Sohbeti Gerçekleştir (Anahtarsız İş Akışı)

Bu kapsamlı bölüm, **anahtarsız temsilci iş akışını** tamamen göstermektedir:

1. **Yapay Zeka Temsilcisi Oluştur**: GPT-4.1 nano modeli ve MCP araçları ile bir temsilci kurun
2. **Konuşma Dizisi Oluştur**: İletişim için bir konuşma dizisi oluşturun
3. **Mesaj Gönder**: Temsilciye Azure OpenAI ile OpenAI arasındaki farkları sorun
4. **Araç Onaylarını Yönet**: Gerekli olduğunda MCP araç çağrılarını otomatik olarak onaylayın
5. **İşlemi İzle**: Temsilcinin ilerlemesini takip edin ve gerekli işlemleri gerçekleştirin
6. **Sonuçları Göster**: Konuşma ve araç kullanım detaylarını görüntüleyin

**Anahtarsız Özellikler:**
- ✅ **Sabitlenmiş sırlar yok** - Tüm kimlik doğrulama Azure kimliği tarafından yönetilir
- ✅ **Varsayılan olarak güvenli** - Rol tabanlı erişim kontrolü kullanır
- ✅ **Basitleştirilmiş dağıtım** - Kimlik bilgisi yönetimi gerekmez
- ✅ **Denetim dostu** - Tüm erişim Azure kimliği üzerinden izlenir

Temsilci, Microsoft Learn kaynaklarına tam güvenlik ve API anahtarı yönetimi olmadan erişmek için MCP araçlarını kullanacaktır.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlık içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul edilmemektedir.
